In [13]:
import pandas as pd

# Data Cleaning

In [14]:
merged_df = pd.read_csv("House Price/Scrap Output.csv")
#Drop null values
merged_df = merged_df.dropna()
#Drop duplicated values
merged_df = merged_df.drop_duplicates(subset='place_name')

C:\Users\PutuAndika\AppData\Local\Temp\ipykernel_13552\3462561239.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df = pd.read_csv("House Price/Scrap Output.csv")


In [15]:
#Assigning kecamatan and city
merged_df[['kecamatan','city']] = merged_df.address.str.split("," ,expand = True)
merged_df = merged_df.drop(columns=['address'])
merged_df.head()

,Unnamed: 0,place_name,bedroom,area,price,mortage,category,date_posted,kecamatan,city
0,0,termurah Ci Ditiro residence low zone,3.0,132 m²,"Rp 5,25 Miliar",Cicilan : 37 Jutaan per bulan,apartemen,"Mimie Diperbarui sejak 06 Juni, 2023",Menteng,Jakarta Pusat
2,1,Dijual murah dan cepat 2 bedroom kosong taman anggrek residences,2.0,50 m²,"Rp 1,5 Miliar",Cicilan : 10 Jutaan per bulan,apartemen,"Alung Diperbarui sejak 05 Juni, 2023",Taman Anggrek,Jakarta Barat
5,2,"Dharmawangsa Residence, Tower 1, 4 Bedroom 350 Sqm, Best Price",4.0,350 m²,Rp 17 Miliar,Cicilan : 121 Jutaan per bulan,apartemen,"Margie SahuburuaDiperbarui sejak 30 Mei, 2023",Kebayoran Baru,Jakarta Selatan
9,3,Apartemen Taman Anggrek Residences Studio Furnish Sudah Sertifikat,1.0,26 m²,Rp 750 Juta,Cicilan : 5 Jutaan per bulan,apartemen,"Marina ZhoDiperbarui sejak 05 Juni, 2023",Taman Anggrek,Jakarta Barat
14,4,Apartement The Mansion Jasmine Tower Bellavista,3.0,119 m²,"Rp 3,2 Miliar",Cicilan : 22 Jutaan per bulan,apartemen,"Sumitro Diperbarui sejak 30 Mei, 2023",Pademangan,Jakarta Utara


## Cleaning Price Column

In [16]:
#Cleaning price column
merged_df[['rp','new_price','number']] = merged_df['price'].str.split(" ", expand = True)
#Function to determine new price
def new_price(a):
    if 'Miliar' in a:
        return 1000000000
    else:
        return 1000000
merged_df['amount'] = merged_df.price.apply(lambda x: new_price(x))
merged_df['price'] = merged_df.price.str.replace("Rp","").str.replace("Miliar", "").str.replace("Juta","").str.replace(",",".").str.strip().astype('float')
merged_df['final_price'] = merged_df['price'] * merged_df['amount']
merged_df = merged_df.drop(columns = ['price', 'new_price', 'rp', 'amount'])

## Cleaning Area

In [17]:
#Cleaning Area
merged_df['area'] = merged_df.area.str.replace("m²","").str.strip().astype('float')

## Cleaning Date Posted

In [18]:
merged_df[['Nama','Date Posted']] = merged_df['date_posted'].str.split("sejak", expand=True)
merged_df['Date Posted'] = merged_df['Date Posted'].str.strip().str.replace(",","").str.strip()
merged_df[['day','month','year']] = merged_df['Date Posted'].str.split(" ", expand=True)
list_month = ['06', '05', '12', '02', '04', '03',
       '01']
merged_df['month'] = merged_df['month'].replace(merged_df['month'].unique(), list_month)
merged_df['date'] = merged_df['day'] + '/' +merged_df['month']+'/'+merged_df['year']
merged_df = merged_df.drop(columns=['Unnamed: 0','Date Posted', 'day', 'month', 'year','date_posted','Nama'])
merged_df.head()

,place_name,bedroom,area,mortage,category,kecamatan,city,number,final_price,date
0,termurah Ci Ditiro residence low zone,3.0,132.0,Cicilan : 37 Jutaan per bulan,apartemen,Menteng,Jakarta Pusat,Miliar,5.250000e+09,06/06/2023
2,Dijual murah dan cepat 2 bedroom kosong taman anggrek residences,2.0,50.0,Cicilan : 10 Jutaan per bulan,apartemen,Taman Anggrek,Jakarta Barat,Miliar,1.500000e+09,05/06/2023
5,"Dharmawangsa Residence, Tower 1, 4 Bedroom 350 Sqm, Best Price",4.0,350.0,Cicilan : 121 Jutaan per bulan,apartemen,Kebayoran Baru,Jakarta Selatan,Miliar,1.700000e+10,30/05/2023
9,Apartemen Taman Anggrek Residences Studio Furnish Sudah Sertifikat,1.0,26.0,Cicilan : 5 Jutaan per bulan,apartemen,Taman Anggrek,Jakarta Barat,Juta,7.500000e+08,05/06/2023
14,Apartement The Mansion Jasmine Tower Bellavista,3.0,119.0,Cicilan : 22 Jutaan per bulan,apartemen,Pademangan,Jakarta Utara,Miliar,3.200000e+09,30/05/2023


## Cleaning Mortage

In [19]:
merged_df['mortage'] = merged_df.mortage.str.replace("Cicilan : ","").str.replace(" per","").str.strip()
merged_df[['number','category','monthly']] = merged_df['mortage'].str.split(" ",expand=True)
merged_df.head()
def mortage_category(a):
    if a == 'Jutaan':
        return 1000000
    elif a == 'Ribuan':
        return 1000
    else:
        return 1000000000
merged_df['number_cat'] = merged_df.category.apply(lambda x: mortage_category(x))
merged_df['number'] = merged_df.number.astype('float')
merged_df['final_mortage_monthly'] = merged_df['number'] * merged_df['number_cat']
merged_df = merged_df.drop(columns=['number_cat', 'number', 'mortage', 'category', 'monthly'])
merged_df.head()

,place_name,bedroom,area,kecamatan,city,final_price,date,final_mortage_monthly
0,termurah Ci Ditiro residence low zone,3.0,132.0,Menteng,Jakarta Pusat,5.250000e+09,06/06/2023,37000000.0
2,Dijual murah dan cepat 2 bedroom kosong taman anggrek residences,2.0,50.0,Taman Anggrek,Jakarta Barat,1.500000e+09,05/06/2023,10000000.0
5,"Dharmawangsa Residence, Tower 1, 4 Bedroom 350 Sqm, Best Price",4.0,350.0,Kebayoran Baru,Jakarta Selatan,1.700000e+10,30/05/2023,121000000.0
9,Apartemen Taman Anggrek Residences Studio Furnish Sudah Sertifikat,1.0,26.0,Taman Anggrek,Jakarta Barat,7.500000e+08,05/06/2023,5000000.0
14,Apartement The Mansion Jasmine Tower Bellavista,3.0,119.0,Pademangan,Jakarta Utara,3.200000e+09,30/05/2023,22000000.0
